# base line

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import imdb
(train_data, train_labels), _ = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results
train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
             loss="binary_crossentropy",
             metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                            epochs=20, batch_size=512, validation_split=0.4)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.6794 - loss: 0.6007 - val_accuracy: 0.8338 - val_loss: 0.4195
Epoch 2/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.8875 - loss: 0.3393 - val_accuracy: 0.8832 - val_loss: 0.3116
Epoch 3/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9232 - loss: 0.2444 - val_accuracy: 0.8861 - val_loss: 0.2856
Epoch 4/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9351 - loss: 0.1970 - val_accuracy: 0.8909 - val_loss: 0.2765
Epoch 5/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9501 - loss: 0.1563 - val_accuracy: 0.8903 - val_loss: 0.2782
Epoch 6/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9614 - loss: 0.1294 - val_accuracy: 0.8856 - val_loss: 0.2878
Epoch 7/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9659 - loss: 0.1145 - val_accuracy: 0.8824 - val_loss: 0.3049
Epoch 8/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accurac

## 사용자 정의 지표

In [3]:
from tensorflow.keras import backend as K

class F1Score(keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super().__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name="tp", initializer="zeros")
        self.false_positives = self.add_weight(name="fp", initializer="zeros")
        self.false_negatives = self.add_weight(name="fn", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = K.cast(y_true, "float32")
        y_pred = K.round(y_pred)

        self.true_positives.assign_add(K.sum(K.cast(y_true * y_pred, "float32")))
        self.false_positives.assign_add(K.sum(K.cast((1 - y_true) * y_pred, "float32")))
        self.false_negatives.assign_add(K.sum(K.cast(y_true * (1 - y_pred), "float32")))

    def result(self):
        precision = self.true_positives / (self.true_positives + self.false_positives + K.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + K.epsilon())

        f1_score = 2 * (precision * recall) / (precision + recall + K.epsilon())
        return f1_score

    def reset_state(self):
        self.true_positives.assign(0)
        self.false_positives.assign(0)
        self.false_negatives.assign(0)

In [4]:
model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy", F1Score()])
n_history = model.fit(train_data, train_labels,
        epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.6681 - f1_score: 0.5916 - loss: 0.6302 - val_accuracy: 0.8567 - val_f1_score: 0.4801 - val_loss: 0.4481
Epoch 2/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8832 - f1_score: 0.5061 - loss: 0.3929 - val_accuracy: 0.8792 - val_f1_score: 0.5128 - val_loss: 0.3406
Epoch 3/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9097 - f1_score: 0.5130 - loss: 0.2858 - val_accuracy: 0.8828 - val_f1_score: 0.4887 - val_loss: 0.3009
Epoch 4/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9242 - f1_score: 0.5052 - loss: 0.2291 - val_accuracy: 0.8901 - val_f1_score: 0.4990 - val_loss: 0.2789
Epoch 5/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9402 - f1_score: 0.5058 - loss: 0.1867 - val_accuracy: 0.8769 - val_f1_score: 0.5311 - val_loss: 0.3034
Epoch 6/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9461 - f1_score: 0.5038 - loss: 0.1646 - val_accuracy: 0.8892 - val_f1_score: 0.5030 - val